In [65]:
from yahoo_fin import options
from yahoo_fin import stock_info as si
import pandas as pd
import numpy as np

ticker = "gme"
# get live price
price = si.get_live_price(ticker)
expo_dates = options.get_expiration_dates(ticker)
chain = options.get_options_chain(ticker)

#options.get_options_chain(ticker, expo_dates[0])

# current options prices (first 4 of them)
puts = options.get_puts(ticker, expo_dates[0]).to_numpy()
options.get_puts(ticker, expo_dates[0])[:3]

<class 'numpy.ndarray'>


,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,GME230106P00005000,2022-12-28 9:59AM EST,5.0,0.01,0.0,0.01,0.00,-,1,37,350.00%
1,GME230106P00010000,2022-12-29 9:51AM EST,10.0,0.02,0.0,0.02,0.00,-,1,201,187.50%
2,GME230106P00011000,2022-12-30 9:52AM EST,11.0,0.03,0.0,0.03,-0.01,-25.00%,2,163,168.75%


The goal is to find if a stock would be profitable if we bought it ITM then immediately sold it, collecting the premium. First, only search options that are ITM.

In [55]:
# remove options that are not currently ITM
itm_puts = [put for put in puts if put[2] > price]
print(itm_puts[0])


['GME230106P00018500' '2022-12-30 3:53PM EST' 18.5 0.79 0.75 0.81 -0.21
 '-21.00%' '114' 293 '74.41%']


cash_out = strike_price
cash_in = premium + price_at_sale

Cost = Strike_price

Gain = Premium + Price at expiration

Profit = premium + price at expiration - strike price

The tricky part is calculating the premium, since it is based on a bid/ask system. I can start with only viewing bid values, as that would guarantee the contract executes, but would cut into profits. If this is successful or no profit is found, I will transition to a mid-point between last price and bid values.

In [1]:

profits = []
for put in itm_puts:
    # cost of buying stock
    cost = put[2] - put[4] # strike - premium
    # gain of selling stock
    gain = price # 3 = last, 4 = bid, 5 = ask
    profits.append((gain - cost)*100)
print(profits)
break_even_price = put[2] - put[4]
print(break_even_price)
print(price)

NameError: name 'itm_puts' is not defined

Zip the current ITM put options with the profits so we can relate them to each other

In [64]:
import itertools
res = [list(itertools.chain(*i))
    for i in zip(itm_puts, profits)]
#test = [j for i in zip(itm_puts, profits) for j in i]
#test = np.dstack((itm_puts, profits))
print(res)

TypeError: 'numpy.float64' object is not iterable